In [140]:
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [141]:
df = pd.read_csv('../data/clean_jsm_titles.csv')
df = df[['clean_title', 'position_level_code']]
df.clean_title = df.clean_title.astype(str)
df.clean_title = df.clean_title.apply(lambda title: title.replace(' ', '_'))

In [142]:
df.head()

,clean_title,position_level_code
0,process_engineer,4
1,receptionist,0
2,dietitian,4
3,engineering_coordinator,3
4,technician,5


In [143]:
corpus = []

for i in range(0,7):
    tmp = df[df.position_level_code == str(i)]
    doc = ' '.join([string for string in tmp.clean_title.values])
    corpus.append(doc)

In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [145]:
tfidf_scores = []

for i in range(len(corpus)):
    vector = vectorizer.transform([corpus[i]]).todense()
    matrix = pd.DataFrame(vector, columns=vectorizer.get_feature_names())
    matrix = matrix.T
    matrix.columns = ['score_' + str(i)]
    tfidf_scores.append(matrix)

In [146]:
# tfidf_scores[0]

In [147]:
df = pd.concat(tfidf_scores, axis=1)

In [148]:
df

,score_0,score_1,score_2,score_3,score_4,score_5,score_6
06,0.000000,0.000000,0.000952,0.000000,0.000000,0.000000,0.000000
0_eh,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000447
0_green_programmer,0.000000,0.000000,0.000000,0.000000,0.000000,0.000401,0.000000
0_lead_technician,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000447
0_tm_e_goop_e_on_job_training,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000447
0_visual_welding_inspector,0.000000,0.000000,0.000000,0.000000,0.000538,0.000000,0.000000
123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000401,0.000000
13,0.000000,0.000000,0.000952,0.000000,0.000000,0.000000,0.000000
130th_flight_engineer_e_cat_c,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000447
15,0.000000,0.000000,0.000952,0.000000,0.000000,0.000000,0.000000


In [95]:
def find_best_category(score_0, score_1, score_2, score_3, score_4, score_5, score_6):
    values = [score_0, score_1, score_2, score_3, score_4, score_5, score_6]
    return values.index(max(values))

df['best_category'] = df.apply(lambda row: find_best_category(row.score_0, row.score_1, row.score_2, row.score_3, row.score_4, row.score_5, row.score_6), axis=1)
df = df['best_category']

06                                                                  2
0_eh                                                                6
0_green_programmer                                                  5
0_lead_technician                                                   6
0_tm_e_goop_e_on_job_training                                       6
0_visual_welding_inspector                                          4
123                                                                 5
13                                                                  2
130th_flight_engineer_e_cat_c                                       6
15                                                                  2
1500                                                                4
15th_trainee_of_islamic_capital_market_graduates_training_scheme    6
17                                                                  4
19                                                                  6
1900                

In [100]:
df = pd.DataFrame(df)
df.reset_index(inplace=True)
df.columns = ['title', 'seniority_category']

In [107]:
df.title = df.title.apply(lambda x : x.replace('_', ' '))

In [126]:
df[df.title.str.match('senior software engineer')]
# print(len(df))
# print(len(df.title.unique()))

,title,seniority_category
68859,senior software engineer,3
68860,senior software engineer and oracle dab,3
68861,senior software engineer bi,3
68862,senior software engineer e development lead,3
68863,senior software engineer e product,3
68864,senior software engineer global,3
68865,senior software engineer p system integrator,3
68866,senior software engineer regional,3
68867,senior software engineer sibelle developer,4
68868,senior software engineer team lead,6
